In [1]:
from signal_generator import get_current_price, get_top10_features, fetch_candlestick_data
from risk_manager import get_current_price, get_instrument_precision, get_current_balance, get_quantity,get_pnl_price,get_open_positions, check_instrument_positions,place_market_order, close_position, calculate_total_unrealised_pnl, close_all_trades
#from notification import send_email_notification
import time
import oandapyV20
import oandapyV20.endpoints.pricing as pricing
import pandas as pd
import numpy as np
import joblib


In [2]:
access_token = "3f0f1b513d44797ff63c09d9da612561-6480f119d75e63cea1e420564d12b6f1"
account_id = "101-003-28603773-001"

accountID = account_id
access_token = access_token
client = oandapyV20.API(access_token=access_token, environment="practice")

In [3]:
#Variable to be changed 

#Status
status = "Active"
#status = "Stop" #close all trades
#status = "Inactive" #leave trades open but inactive 

#Setup
usd_pairs = ["EUR_USD","USD_JPY", "GBP_USD","AUD_USD"]

pairs_mapping = {
    'EUR_USD': 'EURUSD=X',
    'USD_JPY': 'JPY=X',   
    'GBP_USD': 'GBPUSD=X',
    'AUD_USD': 'AUDUSD=X',
    'NZD_USD': 'NZDUSD=X',
    'USD_HKD': 'HKD=X',
    'USD_SGD': 'SGD=X',
    'USD_MXN': 'MXN=X',
    'USD_THB': 'THB=X',
    'USD_ZAR': 'ZAR=X',
}

#Risk management
# risk_factor = 0.016 / 100
# stoploss_pnl = opening_balance * risk_factor
# risk_reward = 0.75  # 3/4
# target_pnl =  stoploss_pnl * risk_reward

# last_print_time = time.time()
# time_interval = 1*60

#Risk reward (1:3) % based on prices
take_profit = 0.015
stop_loss = 0.005

# print("==" * 25)
# print("")
# print("==" * 25)
# print("Starting balance : {:.2f}".format(opening_balance))
# print("Take Profit initial : {:.2f}".format(target_pnl))
# print("Stop loss initial : {:.2f}".format(stoploss_pnl))
# print("==" * 25)

#Library import:
models_15 = joblib.load("models_15mins.joblib")
model_15mins_features = ['DPO_20', 'THERMO_20_2_0.5', 'CFO_9', 'TRUERANGE_1', 'PDIST', 'BBB_5_2.0', 'RVI_14', 'SLOPE_1', 'BULLP_13', 'BOP']

#trade_cycle
trade_cycle = 1
#trade_cycle = 15 * 60 

#market type
order_type = "MARKET"

#Lookback Count
#look back set as 20 to get the latest price action
lookback_count = 20

In [5]:
if status == "Active":       
    try:
        print("Running strategy...")

                
        positions_dict = get_open_positions()

        for instrument in usd_pairs:
            print(instrument)
            price = get_current_price(instrument)
            price_df = fetch_candlestick_data(instrument, lookback_count)
            
            top10_df3 = get_top10_features(price_df["m_open"], price_df["m_high"], price_df["m_low"], price_df["m_close"],model_15mins_features)
            price_df = pd.concat([price_df, top10_df3], axis=1)
            model_data = price_df[model_15mins_features]

            # make prediction for the next period price direction
            # get the currency pair's mode
            model = models_15[pairs_mapping[instrument]]

            # save the predicted y direction
            #   1 is positive => buy t+1, 0 is not positive => sell t+1
            y_pred = model.predict(model_data)
            price_df["prediction_on_next_close"] = [1 if x > 0 else -1 for x in y_pred]
            #find the last signal
            signal = price_df["prediction_on_next_close"].iloc[-1]

            position_type = check_instrument_positions(positions_dict,instrument)
            #
            if position_type == "None": 
                if signal == 1: 
                    quantity = get_quantity(instrument,signal)
                    pnl_price = get_pnl_price(instrument, signal,take_profit,stop_loss)
                    take_profit_price = pnl_price[0]
                    stop_loss_price = pnl_price[1]
                    order = place_market_order(instrument, order_type, quantity, take_profit_price, stop_loss_price)

                elif signal == -1: 
                    quantity = get_quantity(instrument,signal)
                    pnl_price = get_pnl_price(instrument, signal,take_profit,stop_loss)
                    take_profit_price = pnl_price[0]
                    stop_loss_price = pnl_price[1]
                    order = place_market_order(instrument, order_type, quantity, take_profit_price, stop_loss_price)

            elif position_type == "Long":
                if signal == 1: 
                    print ("Hold the trade")
                    
                elif signal == -1: 
                    close_position = close_position(instrument, long_units='ALL', short_units='ALL')
                    quantity = get_quantity(instrument,signal)
                    pnl_price = get_pnl_price(instrument, signal,take_profit,stop_loss)
                    take_profit_price = pnl_price[0]
                    stop_loss_price = pnl_price[1]
                    order = place_market_order(instrument, order_type, quantity, take_profit_price, stop_loss_price)

            elif position_type == "Short":
                if signal == 1: 
                    close_position = close_position(instrument, long_units='ALL', short_units='ALL')
                    quantity = get_quantity(instrument,signal)
                    pnl_price = get_pnl_price(instrument, signal,take_profit,stop_loss)
                    take_profit_price = pnl_price[0]
                    stop_loss_price = pnl_price[1]
                    order = place_market_order(instrument, order_type, quantity, take_profit_price, stop_loss_price)

                elif signal == -1: 
                    print ("Hold the trade")
            else:
                pass  

        time.sleep(trade_cycle)  # Wait for a minute before the next cycle

    except Exception as e:
        print(f"An error occurred: {e}")
        #status = 'Inactive'

elif status == "Inactive":
    
    try: 
        print("Trade is still open")
        positions_dict = get_open_positions()
        print(positions_dict)
        print("Current balance: {:.2f}".format(get_current_balance()))

    except Exception as e:
        print(f"An error occurred: {e}")
        status = 'Stop'

else: 
    print("Closing all Trades")
    close_all_trades(client, accountID)
    print("Current balance: {:.2f}".format(get_current_balance()))


Running strategy...
EUR_USD
{'orderCreateTransaction': {'id': '2070', 'accountID': '101-003-28603773-001', 'userID': 28603773, 'batchID': '2070', 'requestID': '97256799993955113', 'time': '2024-04-13T06:59:42.939262705Z', 'type': 'MARKET_ORDER', 'instrument': 'EUR_USD', 'units': '-100000', 'timeInForce': 'FOK', 'positionFill': 'DEFAULT', 'takeProfitOnFill': {'price': '1.08010', 'timeInForce': 'GTC'}, 'stopLossOnFill': {'price': '1.05880', 'timeInForce': 'GTC', 'triggerMode': 'TOP_OF_BOOK'}, 'reason': 'CLIENT_ORDER'}, 'orderCancelTransaction': {'id': '2071', 'accountID': '101-003-28603773-001', 'userID': 28603773, 'batchID': '2070', 'requestID': '97256799993955113', 'time': '2024-04-13T06:59:42.939262705Z', 'type': 'ORDER_CANCEL', 'orderID': '2070', 'reason': 'MARKET_HALTED'}, 'relatedTransactionIDs': ['2070', '2071'], 'lastTransactionID': '2071'}
Oanda Orders placed successfully!
USD_JPY
{'orderCreateTransaction': {'id': '2072', 'accountID': '101-003-28603773-001', 'userID': 28603773, 

In [140]:
#Fake Order Transaction
succesful_order = {
  "lastTransactionID": "6368",
  "orderCreateTransaction": {
    "accountID": "<ACCOUNT>",
    "batchID": "6367",
    "id": "6367",
    "instrument": "EUR_USD",
    "positionFill": "DEFAULT",
    "reason": "CLIENT_ORDER",
    "time": "2016-06-22T18:41:29.264030555Z",
    "timeInForce": "FOK",
    "type": "MARKET_ORDER",
    "units": "100",
    "userID": "xxx"
  },
  "orderFillTransaction": {
    "accountBalance": "43650.75945",
    "accountID": "<ACCOUNT>",
    "batchID": "6367",
    "financing": "0.00000",
    "id": "6368",
    "instrument": "EUR_USD",
    "orderID": "6367",
    "pl": "0.00000",
    "price": "1.13027",
    "reason": "MARKET_ORDER",
    "time": "2016-06-22T18:41:29.264030555Z",
    "tradeOpened": {
      "tradeID": "6368",
      "units": "100"
    },
    "type": "ORDER_FILL",
    "units": "100",
    "userID": "XXXX"
  },
  "relatedTransactionIDs": [
    "6367",
    "6368"
  ]
}

cancel_order = {'orderCreateTransaction': {'id': '2037',
  'accountID': '101-003-28603773-001',
  'userID': 28603773,
  'batchID': '2037',
  'requestID': '115271160974402940',
  'time': '2024-04-13T04:30:34.739899955Z',
  'type': 'MARKET_ORDER',
  'instrument': 'BTC_USD',
  'units': '1.00',
  'timeInForce': 'FOK',
  'positionFill': 'DEFAULT',
  'takeProfitOnFill': {'price': '68006.4', 'timeInForce': 'GTC'},
  'stopLossOnFill': {'price': '66666.4',
   'timeInForce': 'GTC',
   'triggerMode': 'TOP_OF_BOOK'},
  'reason': 'CLIENT_ORDER'},
 'orderCancelTransaction': {'id': '2038',
  'accountID': '101-003-28603773-001',
  'userID': 28603773,
  'batchID': '2037',
  'requestID': '115271160974402940',
  'time': '2024-04-13T04:30:34.739899955Z',
  'type': 'ORDER_CANCEL',
  'orderID': '2037',
  'reason': 'MARKET_HALTED'},
 'relatedTransactionIDs': ['2037', '2038'],
 'lastTransactionID': '2038'}

In [123]:
#Creating fake position for position
position = {
  "lastTransactionID": "6387", 
  "positions": [
    {
      "instrument": "EUR_USD", 
      "long": {
        "averagePrice": "1.13032", 
        "pl": "-54344.85056", 
        "resettablePL": "-54344.85056", 
        "tradeIDs": [
          "6383", 
          "6385"
        ], 
        "units": "350", 
        "unrealizedPL": "-0.04700"
      }, 
      "pl": "-54300.44169", 
      "resettablePL": "-54300.44169", 
      "short": {
        "pl": "44.40887", 
        "resettablePL": "44.40887", 
        "units": "0", 
        "unrealizedPL": "0.00000"
      }, 
      "unrealizedPL": "-0.04700"
    }, 
    {
      "instrument": "USD_CAD", 
      "long": {
        "pl": "-483.91941", 
        "resettablePL": "-483.91941", 
        "units": "0", 
        "unrealizedPL": "0.00000"
      }, 
      "pl": "-486.16662", 
      "resettablePL": "-486.16662", 
      "short": {
        "averagePrice": "1.28241", 
        "pl": "-2.24721", 
        "resettablePL": "-2.24721", 
        "tradeIDs": [
          "6387"
        ], 
        "units": "-600", 
        "unrealizedPL": "-0.08525"
      }, 
      "unrealizedPL": "-0.08525"
    }
  ]
}

dashboard_position = position